In [1]:
# import airpots2.csv and print head
import pandas as pd
airports = pd.read_csv('data/airports2.csv')

airports.head()

,Origin_airport,Destination_airport,Origin_city,Destination_city,Passengers,Seats,Flights,Distance,Fly_date,Origin_population,Destination_population,Org_airport_lat,Org_airport_long,Dest_airport_lat,Dest_airport_long
0,MHK,AMW,"Manhattan, KS","Ames, IA",21,30,1,254,2008-10-01,122049,86219,39.140999,-96.670799,NaN,NaN
1,EUG,RDM,"Eugene, OR","Bend, OR",41,396,22,103,1990-11-01,284093,76034,44.124599,-123.211998,44.254101,-121.150002
2,EUG,RDM,"Eugene, OR","Bend, OR",88,342,19,103,1990-12-01,284093,76034,44.124599,-123.211998,44.254101,-121.150002
3,EUG,RDM,"Eugene, OR","Bend, OR",11,72,4,103,1990-10-01,284093,76034,44.124599,-123.211998,44.254101,-121.150002
4,MFR,RDM,"Medford, OR","Bend, OR",0,18,1,156,1990-02-01,147300,76034,42.374199,-122.873001,44.254101,-121.150002


In [3]:
# group by destination_airport, of which there may be many entries with the same value, and count the number of flights for each, stored in `flights`. make a new dataframe with one entry per destination_airport, and the sum of the number of flights for each.
flights = airports.groupby('Destination_airport').size()
flights = flights.reset_index(name='flights')

# print the top 10 destination_airports by number of flights
flights.sort_values('flights', ascending=False).head(10)

,Destination_airport,flights
493,ORD,160054
42,ATL,132779
159,DFW,110231
176,DTW,99348
442,MSP,98570
127,CLT,85069
350,LAX,84077
299,IAH,83960
513,PHL,82498
204,EWR,82418


In [9]:
# load airports.json from ../lib/airports.json, load into a df, and print the first 5 entries
import json
with open('data/airports.json') as f:
    airports_json = json.load(f)

airports_json = pd.DataFrame(airports_json).T
airports_json.head()


,icao,iata,name,city,state,country,elevation,lat,lon,tz
00AK,00AK,,Lowell Field,Anchor Point,Alaska,US,450,59.9492,-151.695999,America/Anchorage
00AL,00AL,,Epps Airpark,Harvest,Alabama,US,820,34.864799,-86.770302,America/Chicago
00AZ,00AZ,,Cordes Airport,Cordes,Arizona,US,3810,34.305599,-112.165001,America/Phoenix
00CA,00CA,,Goldstone /Gts/ Airport,Barstow,California,US,3038,35.350498,-116.888,America/Los_Angeles
00CO,00CO,,Cass Field,Briggsdale,Colorado,US,4830,40.6222,-104.344002,America/Denver


In [12]:
# filter to iata not null, print num unique iata codes
airports_json = airports_json[airports_json['iata'].notnull()]
airports_json['iata'].nunique()

7730

In [13]:
# using the flights df, merge with the airports_json df on the Destination_airport column, and discard those who do not have a match in flights

merged = pd.merge(flights, airports_json, left_on='Destination_airport', right_on='iata', how='inner')
merged.head()

,Destination_airport,flights,icao,iata,name,city,state,country,elevation,lat,lon,tz
0,ABE,8353,KABE,ABE,Lehigh Valley International Airport,Allentown,Pennsylvania,US,393,40.6521,-75.440804,America/New_York
1,ABI,2085,KABI,ABI,Abilene Regional Airport,Abilene,Texas,US,1791,32.411301,-99.6819,America/Chicago
2,ABQ,21806,KABQ,ABQ,Albuquerque International Sunport Airport,Albuquerque,New-Mexico,US,5355,35.040199,-106.609001,America/Denver
3,ABR,901,KABR,ABR,Aberdeen Regional Airport,Aberdeen,South-Dakota,US,1302,45.4491,-98.421799,America/Chicago
4,ABY,739,KABY,ABY,Southwest Georgia Regional Airport,Albany,Georgia,US,197,31.5355,-84.194504,America/New_York


In [17]:
# export back to json
merged.to_json('data/merged.json', orient='records')

Process IATA -> desc

In [1]:
from openai import OpenAI
import json
client = OpenAI()

def getBlurb(city: str, client): # city, state format
  response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {
        "role": "system",
        "content": "You will be provided with a city in the United States. Please write a 2-3 sentence descriptive blurb, with the goal of providing an accurate description to potential tourists. Mention the area's climate and key destinations."
      },
      {
        "role": "user",
        "content": city
      }
    ],
    temperature=1,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
  )

  return response.choices[0].message.content

# load data/merged.json
with open('data/merged.json') as f:
    merged = json.load(f)

In [19]:
res = getBlurb('Minneapolis, Minnesota', client)

In [30]:
# create loop to do this for every city in the merged df. run 30 at a time to avoid rate limiting, wait 5 seconds between each batch, and store to a json file
# use try catch to avoid errors, and in the catch block, update a .txt file with the iata codes that failed
# the output json file should have 3 columns: iata, city, and blurb
import time
from tqdm import tqdm
# create failed and blurb list from the json files, if they already exist. otherewise, create empty lists
try:
    with open('data/failed.txt') as f:
        failed = f.read().splitlines()
except Exception as e:
    failed = []
try:
    with open('data/blurb.json') as f:
        blurb = json.load(f)
except Exception as e:
    blurb = []

for i, row in tqdm(enumerate(merged)):
  try:
    # check if iata code is already in blurb list
    if row['iata'] in [b['iata'] for b in blurb]:
      continue
    res = getBlurb(row['city'] + ', ' + row['state'], client)
    blurb.append({'iata': row['iata'], 'city': row['city'], 'blurb': res})
  except Exception as e:
    print(e)
    failed.append(row['iata'])
  if i % 30 == 0:
    time.sleep(5)
    # write to json and failed list in case the process crashes
    with open('data/blurb.json', 'w') as f:
      json.dump(blurb, f)
    with open('data/failed.txt', 'w') as f:
        f.write('\n'.join(failed))

with open('data/blurb.json', 'w') as f:
    json.dump(blurb, f)
with open('data/failed.txt', 'w') as f:
    f.write('\n'.join(failed))

0it [00:00, ?it/s]

649it [25:38,  2.37s/it]


In [2]:
def trimEmbedding(embedding):
    return embedding[:512]

def getEmbedding(client, text):
    res = client.embeddings.create(
        input=text,
        model="text-embedding-3-small"
    )

    return trimEmbedding(res.data[0].embedding)


import os
from supabase import create_client, Client

url: str = os.environ.get("SUPABASE_URL")
key: str = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(url, key)

In [4]:
def addRows(client, table, rows):
    data, count = supabase.table(table).insert(rows).execute()

def processJsonEntryToRow(entry):
    return {
        'iata': entry['iata'],
        'location': entry['city'],
        'blurb': entry['blurb'],
        'embedding': getEmbedding(client, entry['blurb'])
    }

In [14]:
# process all entries in the blurb json file to rows, and add to the supabase table. try 30 at a time to avoid rate limiting, and try catch to avoid errors
# if there is an error, update the failed_emb.txt file with the iata code
from tqdm import tqdm
import time
try:
    with open('data/failed_emb.txt') as f:
        failed_emb = f.read().splitlines()
except Exception as e:
    failed_emb = []

# make a succeeded list as well
try:
    with open('data/succeeded_emb.txt') as f:
        succeeded_emb = f.read().splitlines()
except Exception as e:
    succeeded_emb = []

with open('data/blurb.json') as f:
    blurb = json.load(f)

for i, entry in tqdm(enumerate(blurb)):
    rows = []
    try:
        if entry['iata'] in succeeded_emb:
            continue
        row = processJsonEntryToRow(entry)
        addRows(supabase, 'blurb', [row])
    except Exception as e:
        print(e)
        failed_emb.append(entry['iata'])
    if i % 100 == 0:
        # try:
        #     res = addRows(supabase, 'blurb', rows)
        #     succeeded_emb += [r['iata'] for r in rows]
        # except Exception as e:
        #     failed_emb += [r['iata'] for r in rows]
        # rows = []
        time.sleep(5)
        # with open('data/failed_emb.txt', 'w') as f:
        #     f.write('\n'.join(failed_emb))


0it [00:00, ?it/s]

2024-03-26 14:04:15,574:INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-26 14:04:15,841:INFO - HTTP Request: POST https://nicswsdnhixqjsxkxreg.supabase.co/rest/v1/blurbs "HTTP/1.1 201 Created"
1it [00:05,  5.73s/it]2024-03-26 14:04:21,066:INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-26 14:04:21,329:INFO - HTTP Request: POST https://nicswsdnhixqjsxkxreg.supabase.co/rest/v1/blurbs "HTTP/1.1 201 Created"
2it [00:06,  2.64s/it]2024-03-26 14:04:21,477:INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-26 14:04:21,718:INFO - HTTP Request: POST https://nicswsdnhixqjsxkxreg.supabase.co/rest/v1/blurbs "HTTP/1.1 201 Created"
3it [00:06,  1.61s/it]2024-03-26 14:04:21,875:INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-26 14:04:22,155:INFO - HTTP Request: POST https://nicswsdnhixqjsxkxreg.supabase.co/rest/v1/blurbs "HTTP/1.1 201 Created